In [78]:
import os
import pandas as pd
import numpy as np

In [155]:
# reading the dataset
PImPo = pd.read_csv("PImPo.csv",delimiter=";")
PImPo.rename(columns={"Unnamed: 0":"id",1:"country",2:"party",3:"content",4:"target",5:"stance"},inplace=True)
PImPo.drop(columns=["id"],inplace=True)
PImPo

,country,party,content,target,stance
0,sweden,swe: Green Ecology Party,Vi vill ge människor möjligheten att leva i et...,Integration,FAVOR
1,sweden,swe: Green Ecology Party,och där lika rättigheter för alla är en självk...,Integration,FAVOR
2,sweden,swe: Green Ecology Party,Flyktingpolitiken ska göras human,Immigration,FAVOR
3,sweden,swe: Green Ecology Party,Förslagen i vårt valmanifest grundar sig i en ...,Immigration,FAVOR
4,sweden,swe: Green Ecology Party,"Alla barn har rätt till ett liv i kärlek, resp...",Integration,FAVOR
...,...,...,...,...,...
8955,new zealand,nzl: Maori Party,The Office of the Controller and Auditor Gener...,Integration,NONE
8956,new zealand,nzl: Maori Party,"In return, we expect services to meet the need...",Integration,FAVOR
8957,new zealand,nzl: Maori Party,All new citizens to complete a course in the h...,Integration,NONE
8958,new zealand,nzl: Mana Party,"Maori, Pasifika and migrant peoples are dispro...",Integration,FAVOR


In [156]:
PImPo = PImPo.loc[PImPo["content"].str.split(" ").str.len() > 4]

In [76]:
# use for additional filters
country = "germany"
PImPo = PImPo.loc[PImPo["country"]==country].drop(columns=["country"])

if not os.path.exists("./"+country):
    os.mkdir("./"+country)

In [157]:
# no country
country = "all_notShort"
PImPo = PImPo.drop(columns=["country"])

In [158]:
# use for test/train split
from sklearn.model_selection import train_test_split
PImPo_train, PImPo_test = train_test_split(PImPo,test_size=0.2)

In [159]:
PImPo_train.to_csv(country+"/train.csv",index=False, sep=",")
PImPo_test.to_csv(country+"/test.csv",index=False, sep=",")



In [160]:
data = pd.read_json("all_notShort/data.json")
# print("train:\n",data.loc[data["split"]=="train"]["stance"].value_counts())
# print("valid:\n",data.loc[data["split"]=="valid"]["stance"].value_counts())
# print("test:\n",data.loc[data["split"]=="test"]["stance"].value_counts())
# data = data.loc[data["stance"]=="FAVOR"]
all_df = [len(data["text"].iloc[i]) for i in np.arange(0,len(data))]

pro_df = data.loc[data["stance"]=="FAVOR",:]
pro_df = [len(pro_df["text"].iloc[i]) for i in np.arange(0,len(pro_df))]

against_df = data.loc[data["stance"]=="AGAINST",:]
against_df = [len(against_df["text"].iloc[i]) for i in np.arange(0,len(against_df))]

none_df = data.loc[data["stance"]=="NONE",:]
none_df = [len(none_df["text"].iloc[i]) for i in np.arange(0,len(none_df))]

In [161]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x = all_df,
        opacity=0.75,
        name="ALL"
    )
)
fig.add_trace(
    go.Histogram(
        x = pro_df,
        opacity=0.75,
        name="FAVOR"
    )
)
fig.add_trace(
    go.Histogram(
        x = against_df,
        opacity=0.75,
        name="AGAINST"
    )
)
fig.add_trace(
    go.Histogram(
        x = none_df,
        marker_color='#330C73',
        opacity=0.75,
        name="NONE"
    )
)

fig.update_layout(
    xaxis = dict(
        tick0 = 0,
        dtick = 4
    ),
)

fig.update_layout(
    barmode='overlay',
    title_text = "hist",
    xaxis_title_text='Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2,
    bargroupgap=0.1
)